In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000645531s, CPU 0.000608s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0109904s, CPU 0.010839s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetric)

Problem("body1-2D", :AxiSymmetric, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
supp = FEM.displacementConstraint("left", uy=0)
load = FEM.load("right", fx=1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, 1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [5]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [6]:
FEM.applyBoundaryConditions!(K, f, [supp])

LoadError: applyBoundaryConditions: pdim must be less or equal to 3.

In [7]:
q1 = FEM.solveDisplacement(K, f, support=[supp])

VectorField
[9.482402621954805e-8; 0.0; … ; 2.599460944198026e-6; -9.745989607018536e-7;;]

In [8]:
FEM.showDoFResults(q1, :uvec, visible=true)

1

In [10]:
q2 = FEM.solveDisplacement(problem, load=[load], support=[supp])

VectorField
[9.482402621954805e-8; 0.0; … ; 2.599460944198026e-6; -9.745989607018536e-7;;]

In [11]:
FEM.showDoFResults(q2, :uvec)

2

In [12]:
elsupp = FEM.elasticSupport("left", kx=1e6, ky=1e6)

BoundaryCondition("left", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, 1.0e6, 1.0e6, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [14]:
q4 = FEM.solveDisplacement(problem, load=[load], elasticSupport=[elsupp])

VectorField
[1.5697230533679158e-21; 1.2386799993993293e-5; … ; 2.621375493121948e-6; 9.910734986147553e-6;;]

In [15]:
FEM.showDoFResults(q4, :uvec)

3

In [16]:
q3 = copy(q2)
q3.a[1] = 5
q2.a[1] = 3
q3.a[1]

5.0

In [17]:
q2 + q3
q2 - q3
q2 * 2
2 * q2
q2 / 2

VectorField
[1.5; 0.0; … ; 1.299730472099013e-6; -4.872994803509268e-7;;]

In [18]:
A1 = FEM.solveStrain(q1)

TensorField
[[3.5213212419157086e-6; -1.934496418921024e-8; … ; 0.0; 3.467562206125397e-6;;], [3.5255279209020723e-6; -2.3943325671361946e-8; … ; 0.0; 3.4651516544277885e-6;;], [3.537480773224319e-6; 1.175599164916876e-8; … ; 0.0; 3.4529687493902593e-6;;], [3.4865567547400318e-6; -3.374462459164296e-7; … ; 0.0; 3.2079852751169e-6;;], [3.5319003313561643e-6; 1.00108983869319e-8; … ; 0.0; 3.461646992809151e-6;;], [3.303284510221036e-6; -2.093470612052941e-7; … ; 0.0; 3.0902974050036733e-6;;], [3.53382416854417e-6; -3.5455880042180505e-9; … ; 0.0; 3.4711645943884925e-6;;], [3.5859640773687233e-6; 7.087529149365728e-7; … ; 0.0; 3.092196602683923e-6;;], [3.600701771770717e-6; 1.527950144186748e-7; … ; 0.0; 3.2639366066559133e-6;;], [3.2987608296475336e-6; 7.080164282637666e-7; … ; 0.0; 2.896397398581193e-6;;]  …  [3.6035439452402066e-6; -3.682799929041037e-7; … ; 0.0; 3.4615642064369085e-6;;], [3.502078726794285e-6; -1.8630996573045227e-8; … ; 0.0; 3.4761245835837857e-6;;], [3.5034411912774

In [19]:
A2 = FEM.solveStrain(q1, DoFResults=true)

TensorField
[3.3618534353672486e-6; 6.915791801804869e-6; … ; 0.0; 3.4707402005416507e-6;;]

In [20]:
FEM.showStrainResults(A1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


4

In [21]:
FEM.showDoFResults(A2, :e)

5

In [22]:
A3 = FEM.elementsToNodes(A1)

TensorField
[3.3618534353672486e-6; 3.4578959009024343e-6; … ; 0.0; 3.4707402005416507e-6;;]

In [23]:
FEM.showDoFResults(A3, :e)

6

In [24]:
S1 = FEM.solveStress(q1)

TensorField
[[1.0007816986003317; -0.0029761483368015754; … ; 0.0; 0.9927892182900078;;], [1.00268263877993; -0.003683588564824915; … ; 0.0; 0.9931158418187775;;], [1.00400087992067; 0.001808614099872117; … ; 0.0; 0.9899976943209426;;], [1.0182738526816006; -0.05191480706406609; … ; 0.0; 1.0103602304185717;;], [1.0026292745337215; 0.0015401382133741384; … ; 0.0; 0.9928224043825148;;], [0.9920061557218925; -0.03220724018542986; … ; 0.0; 0.980603067585148;;], [1.001668650761269; -0.0005454750775720078; … ; 0.0; 0.9922543801434881;;], [1.044780676763098; 0.10903890999024196; … ; 0.0; 0.9914086657465208;;], [1.02522089154754; 0.023506925295180735; … ; 0.0; 0.9519103802624245;;], [1.0009319441830153; 0.10892560434827178; … ; 0.0; 0.9164376620071094;;]  …  [1.0491498767439218; -0.05665846044678518; … ; 0.0; 1.0382283583744374;;], [0.9980773700012333; -0.002866307165083881; … ; 0.0; 0.9936236452591045;;], [0.9978082428735044; 0.0028085497698152446; … ; 0.0; 0.9950252883159549;;], [1.194058389

In [25]:
S2 = FEM.solveStress(q1, DoFResults=true)

TensorField
[0.8216722333430639; 0.5319839847542207; … ; 0.0; 0.9941502236238534;;]

In [26]:
FEM.showStressResults(S1, :s)

7

In [27]:
FEM.showDoFResults(S2, :s)

8

In [28]:
S3 = FEM.elementsToNodes(S1)

TensorField
[0.8216722333430639; 0.5319839847542207; … ; 0.0; 0.9941502236238534;;]

In [29]:
FEM.showDoFResults(S3, :s)

9

In [30]:
thermo = FEM.Problem([mat], type=:PlaneHeatConduction)

Problem("body1-2D", :PlaneHeatConduction, 2, 1, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [31]:
temp = FEM.temperatureConstraint("left", T=0)
flux = FEM.heatFlux("right", qn=1)

flx = FEM.heatFluxVector(thermo, [flux])

ScalarField
[0.0; 0.062499999999999924; … ; 0.0; 0.0;;]

In [33]:
KT = FEM.heatConductionMatrix(thermo)

T1 = FEM.solveTemperature(KT, flx, temperatureConstraint=[temp])

ScalarField
[0.0; 0.022222222222222275; … ; 0.0057813518319097855; 0.016643653924583925;;]

In [34]:
FEM.showDoFResults(T1, :T)

10

In [36]:
T2 = FEM.solveTemperature(thermo, heatFlux=[flux], temperatureConstraint=[temp])

ScalarField
[0.0; 0.022222222222222275; … ; 0.0057813518319097855; 0.016643653924583925;;]

In [37]:
FEM.showDoFResults(T2, :T)

11

In [38]:
qT1 = FEM.solveHeatFlux(T2)

VectorField
[[-0.9999999999999996; 4.9960036108132044e-15; … ; -0.9999999999999996; 4.9960036108132044e-15;;], [-0.9999999999999971; -1.2490009027033011e-15; … ; -0.9999999999999971; -1.2490009027033011e-15;;], [-0.9999999999999989; 4.215378046623641e-15; … ; -0.9999999999999989; 4.215378046623641e-15;;], [-1.0000000000000044; -9.367506770274758e-16; … ; -1.0000000000000044; -9.367506770274758e-16;;], [-1.0000000000000002; 1.8735013540549517e-15; … ; -1.0000000000000002; 1.8735013540549517e-15;;], [-1.0000000000000033; -3.903127820947816e-16; … ; -1.0000000000000033; -3.903127820947816e-16;;], [-0.9999999999999989; -2.4980018054066022e-15; … ; -0.9999999999999989; -2.4980018054066022e-15;;], [-1.000000000000003; 3.122502256758253e-16; … ; -1.000000000000003; 3.122502256758253e-16;;], [-1.000000000000003; 1.2490009027033011e-15; … ; -1.000000000000003; 1.2490009027033011e-15;;], [-1.000000000000003; 3.122502256758253e-16; … ; -1.000000000000003; 3.122502256758253e-16;;]  …  [-1.00000000

In [39]:
FEM.showElementResults(qT1, :qvec)

12

In [40]:
qT2 = FEM.solveHeatFlux(T2, DoFResults=true)

VectorField
[-1.000000000000004; -1.5612511283791264e-16; … ; -1.0; -1.5612511283791264e-15;;]

In [41]:
FEM.showDoFResults(qT2, :qvec)

13

In [42]:
qT3 = FEM.elementsToNodes(qT1)

VectorField
[-1.000000000000004; -1.5612511283791264e-16; … ; -1.0; -1.5612511283791264e-15;;]

In [43]:
FEM.showDoFResults(qT3, :qvec)

14

In [45]:
heatconvB = FEM.heatConvection("bottom", h=1, T∞=0)
heatconvT = FEM.heatConvection("top", h=100, T∞=20)

BoundaryCondition("top", nothing, nothing, nothing, nothing, 20, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, 100, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [47]:
T3 = FEM.solveTemperature(thermo, temperatureConstraint=[temp], heatConvection=[heatconvB, heatconvT])

ScalarField
[0.0; 6.028905064944964; … ; 6.149721965107745; 10.661603694875517;;]

In [48]:
FEM.showDoFResults(T3, :T)

15

In [49]:
fT = FEM.thermalLoadVector(problem, T1)

VectorField
[-0.00012298077901520515; -0.00013635384781205732; … ; -0.005981029859801117; -1.3877787807814457e-17;;]

In [51]:
q5 = FEM.solveDisplacement(K, f, support=[supp])

VectorField
[9.482402621954805e-8; 0.0; … ; 2.599460944198026e-6; -9.745989607018536e-7;;]

In [52]:
ST1 = FEM.solveStress(q5, T=T1)

TensorField
[[0.8988260652401932; -0.0029761483368015754; … ; 0.0; 0.8738896416864148;;], [0.883783062176337; -0.003683588564824915; … ; 0.0; 0.891160208458639;;], [0.8868354320476731; 0.001808614099872117; … ; 0.0; 0.8892225149666699;;], [1.0025054946680283; -0.05191480706406609; … ; 0.0; 0.9770268970852384;;], [0.9018540951794488; 0.0015401382133741384; … ; 0.0; 0.8756569565095178;;], [0.9762377977083202; -0.03220724018542986; … ; 0.0; 0.9555935938147241;;], [0.8683353174279356; -0.0005454750775720078; … ; 0.0; 0.8589210468101546;;], [1.030182651137937; 0.10903890999024196; … ; 0.0; 0.9676922364969046;;], [0.9839019777390536; 0.023506925295180735; … ; 0.0; 0.9202346210400415;;], [0.9772155149333992; 0.10892560434827178; … ; 0.0; 0.9018396363819485;;]  …  [1.0241498767439217; -0.05665846044678518; … ; 0.0; 0.9965616917077704;;], [0.8898146594892478; -0.002866307165083881; … ; 0.0; 0.8741702471552896;;], [0.880577514441996; 0.0028085497698152446; … ; 0.0; 0.8872724511401398;;], [1.1690

In [53]:
ST2 = FEM.solveStress(q5, T=T1, T₀=T1)

TensorField
[[1.0007816986003317; -0.0029761483368015754; … ; 0.0; 0.9927892182900078;;], [1.00268263877993; -0.003683588564824915; … ; 0.0; 0.9931158418187775;;], [1.00400087992067; 0.001808614099872117; … ; 0.0; 0.9899976943209426;;], [1.0182738526816006; -0.05191480706406609; … ; 0.0; 1.0103602304185717;;], [1.0026292745337215; 0.0015401382133741384; … ; 0.0; 0.9928224043825148;;], [0.9920061557218925; -0.03220724018542986; … ; 0.0; 0.980603067585148;;], [1.001668650761269; -0.0005454750775720078; … ; 0.0; 0.9922543801434881;;], [1.044780676763098; 0.10903890999024196; … ; 0.0; 0.9914086657465208;;], [1.02522089154754; 0.023506925295180735; … ; 0.0; 0.9519103802624245;;], [1.0009319441830153; 0.10892560434827178; … ; 0.0; 0.9164376620071094;;]  …  [1.0491498767439218; -0.05665846044678518; … ; 0.0; 1.0382283583744374;;], [0.9980773700012333; -0.002866307165083881; … ; 0.0; 0.9936236452591045;;], [0.9978082428735044; 0.0028085497698152446; … ; 0.0; 0.9950252883159549;;], [1.194058389

In [54]:
FEM.showElementResults(ST1, :s)

16

In [55]:
FEM.showElementResults(ST2, :s)

17

In [56]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [57]:
gmsh.finalize()